In [4]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [5]:
import nest_asyncio
nest_asyncio.apply()

In [6]:
text_loader = TextFileLoader("data/KingLear.txt")
documents = text_loader.load_documents()
len(documents)

1

In [7]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

216

In [8]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

In [9]:
vector_db.search_by_text("Your servant Kent. Where is your servant Caius?", k=3)

[(". The same-\n     Your servant Kent. Where is your servant Caius?\n  Lear. He's a good fellow, I can tell you that.\n     He'll strike, and quickly too. He's dead and rotten.\n  Kent. No, my good lord; I am the very man-\n  Lear. I'll see that straight.\n  Kent. That from your first of difference and decay\n     Have followed your sad steps.\n  Lear. You're welcome hither.\n  Kent. Nor no man else! All's cheerless, dark, and deadly.\n     Your eldest daughters have fordone themselves,\n     And desperately are dead.\n  Lear. Ay, so I think.\n  Alb. He knows not what he says; and vain is it\n     That we present us to him.\n  Edg. Very bootless.\n\n                           Enter a Captain. \n\n  Capt. Edmund is dead, my lord.\n  Alb. That's but a trifle here.\n     You lords and noble friends, know our intent.\n     What comfort to this great decay may come\n     Shall be applied. For us, we will resign,\n     During the life of this old Majesty,\n     To him our absolute power; [t

In [10]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
    system_role_prompt.create_message(expertise="Python"),
]

response = chat_openai.run(messages)

In [11]:
RAQA_PROMPT_TEMPLATE = """
Use the provided context to answer the user's query. 

You may not answer the user's query unless there is specific context in the following text.

If you do not know the answer, or cannot answer, please respond with "I don't know".

Context:
{context}
"""

raqa_prompt = SystemRolePrompt(RAQA_PROMPT_TEMPLATE)

USER_PROMPT_TEMPLATE = """
User Query:
{user_query}
"""

user_prompt = UserRolePrompt(USER_PROMPT_TEMPLATE)

class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever

    def run_pipeline(self, user_query: str) -> str:
        context_list = self.vector_db_retriever.search_by_text(user_query, k=4)
        
        context_prompt = ""
        for context in context_list:
            context_prompt += context[0] + "\n"

        formatted_system_prompt = raqa_prompt.create_message(context=context_prompt)

        formatted_user_prompt = user_prompt.create_message(user_query=user_query)
        
        return self.llm.run([formatted_system_prompt, formatted_user_prompt])

In [12]:
retrieval_augmented_qa_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai
)

In [13]:
retrieval_augmented_qa_pipeline.run_pipeline("Who is King Lear?")

'King Lear is the main character in William Shakespeare\'s play, "King Lear." He is the King of Britain and the father of three daughters: Goneril, Regan, and Cordelia. The play focuses on Lear\'s descent into madness and the consequences of his decision to divide his kingdom between his two elder daughters, who betray and mistreat him.'

In [14]:
retrieval_augmented_qa_pipeline.run_pipeline("Who is Batman?")

"I don't know."

In [21]:
retrieval_augmented_qa_pipeline.run_pipeline("What happens to Cordelia?")

"Cordelia is seen in the French camp with Kent, the Doctor, and a Gentleman. They discuss Cordelia's feelings towards Kent and her gratitude for his goodness. They mention that Cordelia's life will be too short to repay Kent's goodness. The scene does not provide any specific information about what happens to Cordelia after this moment."